In [41]:
import boto3
import json
import pandas as pd
s3 = boto3.resource("s3")
bucket = s3.Bucket('auto-shorts')

In [28]:
all_data = {}
for obj in bucket.objects.all():
    target = "tmp_data.json"
    if 'video_data.json' in obj.key:
        bucket.download_file(obj.key, target)
        with open('tmp_data.json', 'r') as f:
            vid_data = json.load(f)
            vid_path = '/'.join(obj.key.split('/')[:-1])
            vid_len = vid_data['most_watched_moments'][-1]['time_end_ms']/1000
            all_data[vid_path] = vid_len

In [ ]:
df = pd.DataFrame(list(all_data.items()), columns=['s3_path', 'vid_len'])

In [ ]:
from sqlalchemy import URL, create_engine,text

postgres_engine = create_engine(
    URL.create(
        host=POSTGRES_HOST,
        drivername="postgresql+psycopg2",
        username=POSTGRES_USERNAME,
        password=POSTGRES_PASSWORD,
        database=POSTGRES_DB,
        port=5432,
    ),
    pool_pre_ping=True,
)

In [ ]:
with postgres_engine.connect() as conn:
    df_vids = pd.read_sql_query(text('SELECT * FROM autoshorts.videos'), con=conn)[['s3_path', 'category_id']]

In [ ]:
final_df = df.merge(df_vids, on='s3_path')

In [ ]:
final_df.groupby('category_id')['vid_len'].agg(['min', 'max', 'std', 'mean', 'count', 'median']).T